# In this notebook, I followed this [tutorial](https://www.youtube.com/watch?v=EoGUlvhRYpk&list=PLEcgPc5tI9rYQSwZEsDu8fk6mBGLX2Bi2&index=13) from youtube.

In [70]:
import torch
import torch.nn as nn
import torch.optim as optim
#from torch.utils.tensorboard import SummaryWriter   # to print tensorboard | to get nice cost plot

# from utils2 import translate_sentence, bleu, save_checkpoint, load_checkpoint
# translate_sentence, bleu in 2non ko spacy and torch text chahiye thi, vo meray pas version k bht zida conflicts hain, so ye kaheen oor say haasil krain gay
# save_checkpoint, load_checkpoint, in 2non ki definitions main nay next cell main paste kr di hain


# import torchtext; torchtext.disable_torchtext_deprecation_warning()
# from torchtext.datasets import Multi30k                # Multi30k dataset: A german to english dataset
#from torchtext.data import Field, BucketIterator      # For the preprocessing of data
# from datasets import load_dataset                       # dataset = load_dataset("wmt16", "de-en")
import tarfile
import nltk
# set nltk path
nltk.data.path.append('/nltk_data')
from nltk import Counter
import re

import numpy as np
#import spacy                                          # For tokenization
import random



In [72]:
torch.cuda.is_available()


True

In [73]:
torch.__version__


'1.9.0'

# Importing Data


In [74]:
# import tarfile

# Open the tar.gz file
with tarfile.open('data/val/validation.tar.gz', 'r:gz') as tar:
    # Extract all contents to the current directory
    tar.extractall()

print("Extraction complete!")

# val.de etc


Extraction complete!


In [75]:
# Open the tar.gz file
with tarfile.open('data/train/training.tar.gz', 'r:gz') as tar:
    # Extract all contents to the current directory
    tar.extractall()

print("Extraction complete!")
# train.de etc


Extraction complete!


In [76]:
# Open the tar.gz file
with tarfile.open('data/test/mmt16_task1_test.tar.gz', 'r:gz') as tar:
    # Extract all contents to the current directory
    tar.extractall()

print("Extraction complete!")


Extraction complete!


In [77]:
with open('data/train-en.txt', 'r') as f:
    train_en = f.readlines()
train_en

['Two young, White males are outside near many bushes.\n',
 'Several men in hard hats are operating a giant pulley system.\n',
 'A little girl climbing into a wooden playhouse.\n',
 'A man in a blue shirt is standing on a ladder cleaning a window.\n',
 'Two men are at the stove preparing food.\n',
 'A man in green holds a guitar while the other man observes his shirt.\n',
 'A man is smiling at a stuffed lion\n',
 'A trendy girl talking on her cellphone while gliding slowly down the street.\n',
 'A woman with a large purse is walking by a gate.\n',
 'Boys dancing on poles in the middle of the night.\n',
 'A ballet class of five girls jumping in sequence.\n',
 'Four guys three wearing hats one not are jumping at the top of a staircase.\n',
 'A black dog and a spotted dog are fighting\n',
 'A man in a neon green and orange uniform is driving on a green tractor.\n',
 'Several women wait outside in a city.\n',
 'A lady in a black top with glasses is sprinkling powdered sugar on a bundt cake

In [78]:
train_en = [re.sub(r'.\n', '', sentence.lower()) for sentence in train_en]
train_en


['two young, white males are outside near many bushes',
 'several men in hard hats are operating a giant pulley system',
 'a little girl climbing into a wooden playhouse',
 'a man in a blue shirt is standing on a ladder cleaning a window',
 'two men are at the stove preparing food',
 'a man in green holds a guitar while the other man observes his shirt',
 'a man is smiling at a stuffed lio',
 'a trendy girl talking on her cellphone while gliding slowly down the street',
 'a woman with a large purse is walking by a gate',
 'boys dancing on poles in the middle of the night',
 'a ballet class of five girls jumping in sequence',
 'four guys three wearing hats one not are jumping at the top of a staircase',
 'a black dog and a spotted dog are fightin',
 'a man in a neon green and orange uniform is driving on a green tractor',
 'several women wait outside in a city',
 'a lady in a black top with glasses is sprinkling powdered sugar on a bundt cake',
 'a little girl is sitting in front of a l

In [79]:
len(train_en)


29001

In [80]:
# Try with UTF-8 encoding
with open('data/train-de.txt', 'r', encoding='utf-8') as f:
    train_de = f.readlines()
train_de

['Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.\n',
 'Mehrere Männer mit Schutzhelmen bedienen ein Antriebsradsystem.\n',
 'Ein kleines Mädchen klettert in ein Spielhaus aus Holz.\n',
 'Ein Mann in einem blauen Hemd steht auf einer Leiter und putzt ein Fenster.\n',
 'Zwei Männer stehen am Herd und bereiten Essen zu.\n',
 'Ein Mann in grün hält eine Gitarre, während der andere Mann sein Hemd ansieht.\n',
 'Ein Mann lächelt einen ausgestopften Löwen an.\n',
 'Ein schickes Mädchen spricht mit dem Handy während sie langsam die Straße entlangschwebt.\n',
 'Eine Frau mit einer großen Geldbörse geht an einem Tor vorbei.\n',
 'Jungen tanzen mitten in der Nacht auf Pfosten.\n',
 'Eine Ballettklasse mit fünf Mädchen, die nacheinander springen.\n',
 'Vier Typen, von denen drei Hüte tragen und einer nicht, springen oben in einem Treppenhaus.\n',
 'Ein schwarzer Hund und ein gefleckter Hund kämpfen.\n',
 'Ein Mann in einer neongrünen und orangefarbenen Uniform fährt auf einem gr

In [81]:
train_de = [re.sub(r'.\n', '', sentence.lower()) for sentence in train_de]
train_de


['zwei junge weiße männer sind im freien in der nähe vieler büsche',
 'mehrere männer mit schutzhelmen bedienen ein antriebsradsystem',
 'ein kleines mädchen klettert in ein spielhaus aus holz',
 'ein mann in einem blauen hemd steht auf einer leiter und putzt ein fenster',
 'zwei männer stehen am herd und bereiten essen zu',
 'ein mann in grün hält eine gitarre, während der andere mann sein hemd ansieht',
 'ein mann lächelt einen ausgestopften löwen an',
 'ein schickes mädchen spricht mit dem handy während sie langsam die straße entlangschwebt',
 'eine frau mit einer großen geldbörse geht an einem tor vorbei',
 'jungen tanzen mitten in der nacht auf pfosten',
 'eine ballettklasse mit fünf mädchen, die nacheinander springen',
 'vier typen, von denen drei hüte tragen und einer nicht, springen oben in einem treppenhaus',
 'ein schwarzer hund und ein gefleckter hund kämpfen',
 'ein mann in einer neongrünen und orangefarbenen uniform fährt auf einem grünen traktor',
 'mehrere frauen warten 

In [82]:
len(train_de)


29001

# Preprocessing Data


In [83]:
print(nltk.word_tokenize(train_en[0].lower(), language='english'))


['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes']


In [84]:
train_de[0].lower()


'zwei junge weiße männer sind im freien in der nähe vieler büsche'

In [85]:
print(nltk.word_tokenize(train_de[0].lower(), language='german'))


['zwei', 'junge', 'weiße', 'männer', 'sind', 'im', 'freien', 'in', 'der', 'nähe', 'vieler', 'büsche']


In [86]:
train_en

['two young, white males are outside near many bushes',
 'several men in hard hats are operating a giant pulley system',
 'a little girl climbing into a wooden playhouse',
 'a man in a blue shirt is standing on a ladder cleaning a window',
 'two men are at the stove preparing food',
 'a man in green holds a guitar while the other man observes his shirt',
 'a man is smiling at a stuffed lio',
 'a trendy girl talking on her cellphone while gliding slowly down the street',
 'a woman with a large purse is walking by a gate',
 'boys dancing on poles in the middle of the night',
 'a ballet class of five girls jumping in sequence',
 'four guys three wearing hats one not are jumping at the top of a staircase',
 'a black dog and a spotted dog are fightin',
 'a man in a neon green and orange uniform is driving on a green tractor',
 'several women wait outside in a city',
 'a lady in a black top with glasses is sprinkling powdered sugar on a bundt cake',
 'a little girl is sitting in front of a l

In [87]:
'a ' + train_en[0] + ' b'


'a two young, white males are outside near many bushes b'

In [88]:
['<sos>'] + nltk.word_tokenize(train_en[0], language='english') + ['<eos>']


['<sos>',
 'two',
 'young',
 ',',
 'white',
 'males',
 'are',
 'outside',
 'near',
 'many',
 'bushes',
 '<eos>']

In [89]:
for i, string in enumerate(train_en):
    train_en[i] = ['<sos>'] + nltk.word_tokenize(string, language='english') + ['<eos>']
train_en[0]


['<sos>',
 'two',
 'young',
 ',',
 'white',
 'males',
 'are',
 'outside',
 'near',
 'many',
 'bushes',
 '<eos>']

In [90]:
print(train_en[2])


['<sos>', 'a', 'little', 'girl', 'climbing', 'into', 'a', 'wooden', 'playhouse', '<eos>']


In [91]:
for i, string in enumerate(train_de):
    train_de[i] = ['<sos>'] + nltk.word_tokenize(string, language='german') + ['<eos>']
print(train_de[0])


['<sos>', 'zwei', 'junge', 'weiße', 'männer', 'sind', 'im', 'freien', 'in', 'der', 'nähe', 'vieler', 'büsche', '<eos>']


In [92]:
# building english vocab

vocab_en   = {'<unk>':0, '<pad>':1, '<sos>':2, '<eos>':3}
vocab_freq_en = {}

for list_of_tokens in train_en:
    for token in list_of_tokens:
        if token in vocab_freq_en.keys():
            vocab_freq_en[token] += 1
        else:
            vocab_freq_en[token] = 1

for key, value in vocab_freq_en.items():
    if key not in ['<unk>', '<pad>', '<sos>', '<eos>']:
        if value >= 2:
            vocab_en[key] = len(vocab_en)

vocab_en


{'<unk>': 0,
 '<pad>': 1,
 '<sos>': 2,
 '<eos>': 3,
 'two': 4,
 'young': 5,
 ',': 6,
 'white': 7,
 'males': 8,
 'are': 9,
 'outside': 10,
 'near': 11,
 'many': 12,
 'bushes': 13,
 'several': 14,
 'men': 15,
 'in': 16,
 'hard': 17,
 'hats': 18,
 'operating': 19,
 'a': 20,
 'giant': 21,
 'pulley': 22,
 'system': 23,
 'little': 24,
 'girl': 25,
 'climbing': 26,
 'into': 27,
 'wooden': 28,
 'playhouse': 29,
 'man': 30,
 'blue': 31,
 'shirt': 32,
 'is': 33,
 'standing': 34,
 'on': 35,
 'ladder': 36,
 'cleaning': 37,
 'window': 38,
 'at': 39,
 'the': 40,
 'stove': 41,
 'preparing': 42,
 'food': 43,
 'green': 44,
 'holds': 45,
 'guitar': 46,
 'while': 47,
 'other': 48,
 'observes': 49,
 'his': 50,
 'smiling': 51,
 'stuffed': 52,
 'trendy': 53,
 'talking': 54,
 'her': 55,
 'cellphone': 56,
 'gliding': 57,
 'slowly': 58,
 'down': 59,
 'street': 60,
 'woman': 61,
 'with': 62,
 'large': 63,
 'purse': 64,
 'walking': 65,
 'by': 66,
 'gate': 67,
 'boys': 68,
 'dancing': 69,
 'poles': 70,
 'middle':

In [93]:
len(vocab_en)

6084

In [94]:
vocab_en['<pad>'], vocab_en['<unk>'], len(vocab_en)


(1, 0, 6084)

In [95]:
index_to_word_en = {}

for word, index in vocab_en.items():
    index_to_word_en[index] = word
index_to_word_en


{0: '<unk>',
 1: '<pad>',
 2: '<sos>',
 3: '<eos>',
 4: 'two',
 5: 'young',
 6: ',',
 7: 'white',
 8: 'males',
 9: 'are',
 10: 'outside',
 11: 'near',
 12: 'many',
 13: 'bushes',
 14: 'several',
 15: 'men',
 16: 'in',
 17: 'hard',
 18: 'hats',
 19: 'operating',
 20: 'a',
 21: 'giant',
 22: 'pulley',
 23: 'system',
 24: 'little',
 25: 'girl',
 26: 'climbing',
 27: 'into',
 28: 'wooden',
 29: 'playhouse',
 30: 'man',
 31: 'blue',
 32: 'shirt',
 33: 'is',
 34: 'standing',
 35: 'on',
 36: 'ladder',
 37: 'cleaning',
 38: 'window',
 39: 'at',
 40: 'the',
 41: 'stove',
 42: 'preparing',
 43: 'food',
 44: 'green',
 45: 'holds',
 46: 'guitar',
 47: 'while',
 48: 'other',
 49: 'observes',
 50: 'his',
 51: 'smiling',
 52: 'stuffed',
 53: 'trendy',
 54: 'talking',
 55: 'her',
 56: 'cellphone',
 57: 'gliding',
 58: 'slowly',
 59: 'down',
 60: 'street',
 61: 'woman',
 62: 'with',
 63: 'large',
 64: 'purse',
 65: 'walking',
 66: 'by',
 67: 'gate',
 68: 'boys',
 69: 'dancing',
 70: 'poles',
 71: 'midd

In [96]:
############################################################################

In [97]:
if 'a' not in {'q':1, 'w':2, 'e':3}.keys():
    print('a')
else:
    print('b')


a


In [98]:
if 'a' not in {'a':1, 'w':2, 'e':3}.keys():
    print('a')
else:
    print('b')


b


In [99]:
############################################################################

In [100]:
# building german vocab

vocab_de   = {'<unk>':0, '<pad>':1, '<sos>':2, '<eos>':3}
vocab_freq_de = {}

for list_of_tokens in train_de:
    for token in list_of_tokens:
        if token in vocab_freq_de.keys():
            vocab_freq_de[token] += 1
        else:
            vocab_freq_de[token] = 1

for key, value in vocab_freq_de.items():
    if key not in ['<unk>', '<pad>', '<sos>', '<eos>']:
        if value >= 2:
            vocab_de[key] = len(vocab_de)

vocab_de


{'<unk>': 0,
 '<pad>': 1,
 '<sos>': 2,
 '<eos>': 3,
 'zwei': 4,
 'junge': 5,
 'weiße': 6,
 'männer': 7,
 'sind': 8,
 'im': 9,
 'freien': 10,
 'in': 11,
 'der': 12,
 'nähe': 13,
 'vieler': 14,
 'büsche': 15,
 'mehrere': 16,
 'mit': 17,
 'schutzhelmen': 18,
 'bedienen': 19,
 'ein': 20,
 'kleines': 21,
 'mädchen': 22,
 'klettert': 23,
 'spielhaus': 24,
 'aus': 25,
 'holz': 26,
 'mann': 27,
 'einem': 28,
 'blauen': 29,
 'hemd': 30,
 'steht': 31,
 'auf': 32,
 'einer': 33,
 'leiter': 34,
 'und': 35,
 'putzt': 36,
 'fenster': 37,
 'stehen': 38,
 'am': 39,
 'herd': 40,
 'bereiten': 41,
 'essen': 42,
 'zu': 43,
 'grün': 44,
 'hält': 45,
 'eine': 46,
 'gitarre': 47,
 ',': 48,
 'während': 49,
 'andere': 50,
 'sein': 51,
 'ansieht': 52,
 'lächelt': 53,
 'einen': 54,
 'ausgestopften': 55,
 'löwen': 56,
 'an': 57,
 'schickes': 58,
 'spricht': 59,
 'dem': 60,
 'handy': 61,
 'sie': 62,
 'langsam': 63,
 'die': 64,
 'straße': 65,
 'frau': 66,
 'großen': 67,
 'geldbörse': 68,
 'geht': 69,
 'tor': 70,
 'v

In [101]:
vocab_freq_de


{'<sos>': 29001,
 'zwei': 3872,
 'junge': 2068,
 'weiße': 146,
 'männer': 1662,
 'sind': 489,
 'im': 3107,
 'freien': 474,
 'in': 11893,
 'der': 4989,
 'nähe': 383,
 'vieler': 2,
 'büsche': 5,
 '<eos>': 29001,
 'mehrere': 580,
 'mit': 8843,
 'schutzhelmen': 33,
 'bedienen': 9,
 'ein': 18850,
 'antriebsradsystem': 1,
 'kleines': 772,
 'mädchen': 2121,
 'klettert': 171,
 'spielhaus': 2,
 'aus': 910,
 'holz': 61,
 'mann': 7805,
 'einem': 13711,
 'blauen': 992,
 'hemd': 1202,
 'steht': 1774,
 'auf': 8745,
 'einer': 6765,
 'leiter': 56,
 'und': 8925,
 'putzt': 43,
 'fenster': 148,
 'stehen': 940,
 'am': 911,
 'herd': 17,
 'bereiten': 79,
 'essen': 373,
 'zu': 1910,
 'grün': 58,
 'hält': 1308,
 'eine': 9908,
 'gitarre': 303,
 ',': 8935,
 'während': 1397,
 'andere': 412,
 'sein': 292,
 'ansieht': 12,
 'lächelt': 234,
 'einen': 3479,
 'ausgestopften': 3,
 'löwen': 3,
 'an': 3061,
 'schickes': 4,
 'spricht': 211,
 'dem': 2134,
 'handy': 129,
 'sie': 429,
 'langsam': 4,
 'die': 3948,
 'straße': 

In [102]:
vocab_de['<pad>'], vocab_de['<unk>'], len(vocab_de)


(1, 0, 7875)

In [103]:
index_to_word_de = {}

for word, index in vocab_de.items():
    index_to_word_de[index] = word
index_to_word_de


{0: '<unk>',
 1: '<pad>',
 2: '<sos>',
 3: '<eos>',
 4: 'zwei',
 5: 'junge',
 6: 'weiße',
 7: 'männer',
 8: 'sind',
 9: 'im',
 10: 'freien',
 11: 'in',
 12: 'der',
 13: 'nähe',
 14: 'vieler',
 15: 'büsche',
 16: 'mehrere',
 17: 'mit',
 18: 'schutzhelmen',
 19: 'bedienen',
 20: 'ein',
 21: 'kleines',
 22: 'mädchen',
 23: 'klettert',
 24: 'spielhaus',
 25: 'aus',
 26: 'holz',
 27: 'mann',
 28: 'einem',
 29: 'blauen',
 30: 'hemd',
 31: 'steht',
 32: 'auf',
 33: 'einer',
 34: 'leiter',
 35: 'und',
 36: 'putzt',
 37: 'fenster',
 38: 'stehen',
 39: 'am',
 40: 'herd',
 41: 'bereiten',
 42: 'essen',
 43: 'zu',
 44: 'grün',
 45: 'hält',
 46: 'eine',
 47: 'gitarre',
 48: ',',
 49: 'während',
 50: 'andere',
 51: 'sein',
 52: 'ansieht',
 53: 'lächelt',
 54: 'einen',
 55: 'ausgestopften',
 56: 'löwen',
 57: 'an',
 58: 'schickes',
 59: 'spricht',
 60: 'dem',
 61: 'handy',
 62: 'sie',
 63: 'langsam',
 64: 'die',
 65: 'straße',
 66: 'frau',
 67: 'großen',
 68: 'geldbörse',
 69: 'geht',
 70: 'tor',
 71

# Converting Word sequences into Number Sequences

In [104]:
def word_seq_to_int_seq(word_seq_, vocab_):
    int_seq = []
    for word in word_seq_:
        current_int = vocab_.get(word, vocab_['<unk>'])
        int_seq.append(current_int)
    return int_seq


In [105]:
word_seq_to_int_seq(['my', 'name', 'is', 'ans', 'imran'], vocab_en)
# perfect


[2841, 1849, 33, 0, 0]

In [106]:
# English conversions
train_en_int = []
for current_word_seq in train_en:
    current_int_seq = word_seq_to_int_seq(current_word_seq, vocab_en)
    train_en_int.append(current_int_seq)

train_en_int


[[2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 3],
 [2, 14, 15, 16, 17, 18, 9, 19, 20, 21, 22, 23, 3],
 [2, 20, 24, 25, 26, 27, 20, 28, 29, 3],
 [2, 20, 30, 16, 20, 31, 32, 33, 34, 35, 20, 36, 37, 20, 38, 3],
 [2, 4, 15, 9, 39, 40, 41, 42, 43, 3],
 [2, 20, 30, 16, 44, 45, 20, 46, 47, 40, 48, 30, 49, 50, 32, 3],
 [2, 20, 30, 33, 51, 39, 20, 52, 0, 3],
 [2, 20, 53, 25, 54, 35, 55, 56, 47, 57, 58, 59, 40, 60, 3],
 [2, 20, 61, 62, 20, 63, 64, 33, 65, 66, 20, 67, 3],
 [2, 68, 69, 35, 70, 16, 40, 71, 72, 40, 73, 3],
 [2, 20, 74, 75, 72, 76, 77, 78, 16, 79, 3],
 [2, 80, 81, 82, 83, 18, 84, 85, 9, 78, 39, 40, 86, 72, 20, 87, 3],
 [2, 20, 88, 89, 90, 20, 91, 89, 9, 0, 3],
 [2, 20, 30, 16, 20, 92, 44, 90, 93, 94, 33, 95, 35, 20, 44, 96, 3],
 [2, 14, 97, 98, 10, 16, 20, 99, 3],
 [2, 20, 100, 16, 20, 88, 86, 62, 101, 33, 102, 103, 0, 35, 20, 0, 104, 3],
 [2, 20, 24, 25, 33, 105, 16, 106, 72, 20, 63, 107, 108, 3],
 [2, 20, 30, 109, 35, 40, 110, 111, 112, 20, 7, 89, 33, 113, 114, 3],
 [2, 76, 115, 9, 105, 1

In [107]:
print(train_en[0])


['<sos>', 'two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '<eos>']


In [108]:
print(train_en_int[0])


[2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 3]


In [109]:
vocab_en
# perfect


{'<unk>': 0,
 '<pad>': 1,
 '<sos>': 2,
 '<eos>': 3,
 'two': 4,
 'young': 5,
 ',': 6,
 'white': 7,
 'males': 8,
 'are': 9,
 'outside': 10,
 'near': 11,
 'many': 12,
 'bushes': 13,
 'several': 14,
 'men': 15,
 'in': 16,
 'hard': 17,
 'hats': 18,
 'operating': 19,
 'a': 20,
 'giant': 21,
 'pulley': 22,
 'system': 23,
 'little': 24,
 'girl': 25,
 'climbing': 26,
 'into': 27,
 'wooden': 28,
 'playhouse': 29,
 'man': 30,
 'blue': 31,
 'shirt': 32,
 'is': 33,
 'standing': 34,
 'on': 35,
 'ladder': 36,
 'cleaning': 37,
 'window': 38,
 'at': 39,
 'the': 40,
 'stove': 41,
 'preparing': 42,
 'food': 43,
 'green': 44,
 'holds': 45,
 'guitar': 46,
 'while': 47,
 'other': 48,
 'observes': 49,
 'his': 50,
 'smiling': 51,
 'stuffed': 52,
 'trendy': 53,
 'talking': 54,
 'her': 55,
 'cellphone': 56,
 'gliding': 57,
 'slowly': 58,
 'down': 59,
 'street': 60,
 'woman': 61,
 'with': 62,
 'large': 63,
 'purse': 64,
 'walking': 65,
 'by': 66,
 'gate': 67,
 'boys': 68,
 'dancing': 69,
 'poles': 70,
 'middle':

In [110]:
# German conversions
train_de_int = []
for current_word_seq in train_de:
    current_int_seq = word_seq_to_int_seq(current_word_seq, vocab_de)
    train_de_int.append(current_int_seq)

train_de_int


[[2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 3],
 [2, 16, 7, 17, 18, 19, 20, 0, 3],
 [2, 20, 21, 22, 23, 11, 20, 24, 25, 26, 3],
 [2, 20, 27, 11, 28, 29, 30, 31, 32, 33, 34, 35, 36, 20, 37, 3],
 [2, 4, 7, 38, 39, 40, 35, 41, 42, 43, 3],
 [2, 20, 27, 11, 44, 45, 46, 47, 48, 49, 12, 50, 27, 51, 30, 52, 3],
 [2, 20, 27, 53, 54, 55, 56, 57, 3],
 [2, 20, 58, 22, 59, 17, 60, 61, 49, 62, 63, 64, 65, 0, 3],
 [2, 46, 66, 17, 33, 67, 68, 69, 57, 28, 70, 71, 3],
 [2, 72, 73, 74, 11, 12, 75, 32, 76, 3],
 [2, 46, 77, 17, 78, 22, 48, 64, 79, 80, 3],
 [2, 81, 82, 48, 83, 84, 85, 86, 87, 35, 33, 88, 48, 80, 89, 11, 28, 90, 3],
 [2, 20, 91, 92, 35, 20, 93, 92, 94, 3],
 [2, 20, 27, 11, 33, 95, 35, 96, 97, 98, 32, 28, 99, 100, 3],
 [2, 16, 101, 102, 11, 33, 103, 9, 10, 3],
 [2, 46, 66, 17, 104, 105, 35, 106, 107, 108, 32, 28, 0, 3],
 [2, 20, 21, 22, 109, 110, 28, 67, 111, 112, 3],
 [2, 20, 27, 113, 32, 12, 114, 48, 57, 64, 115, 20, 116, 92, 117, 118, 3],
 [2, 78, 119, 120, 17, 121, 9, 122, 3],
 [2, 46,

In [111]:
print(train_de[0])


['<sos>', 'zwei', 'junge', 'weiße', 'männer', 'sind', 'im', 'freien', 'in', 'der', 'nähe', 'vieler', 'büsche', '<eos>']


In [112]:
print(train_de_int[0])


[2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 3]


In [113]:
vocab_de
# perfect


{'<unk>': 0,
 '<pad>': 1,
 '<sos>': 2,
 '<eos>': 3,
 'zwei': 4,
 'junge': 5,
 'weiße': 6,
 'männer': 7,
 'sind': 8,
 'im': 9,
 'freien': 10,
 'in': 11,
 'der': 12,
 'nähe': 13,
 'vieler': 14,
 'büsche': 15,
 'mehrere': 16,
 'mit': 17,
 'schutzhelmen': 18,
 'bedienen': 19,
 'ein': 20,
 'kleines': 21,
 'mädchen': 22,
 'klettert': 23,
 'spielhaus': 24,
 'aus': 25,
 'holz': 26,
 'mann': 27,
 'einem': 28,
 'blauen': 29,
 'hemd': 30,
 'steht': 31,
 'auf': 32,
 'einer': 33,
 'leiter': 34,
 'und': 35,
 'putzt': 36,
 'fenster': 37,
 'stehen': 38,
 'am': 39,
 'herd': 40,
 'bereiten': 41,
 'essen': 42,
 'zu': 43,
 'grün': 44,
 'hält': 45,
 'eine': 46,
 'gitarre': 47,
 ',': 48,
 'während': 49,
 'andere': 50,
 'sein': 51,
 'ansieht': 52,
 'lächelt': 53,
 'einen': 54,
 'ausgestopften': 55,
 'löwen': 56,
 'an': 57,
 'schickes': 58,
 'spricht': 59,
 'dem': 60,
 'handy': 61,
 'sie': 62,
 'langsam': 63,
 'die': 64,
 'straße': 65,
 'frau': 66,
 'großen': 67,
 'geldbörse': 68,
 'geht': 69,
 'tor': 70,
 'v

# Data Generator

In [114]:
def data_generator(train_en_int_, train_de_int_, vocab_en_, vocab_de_, batch_size_, pad_key_de=vocab_de['<pad>'], pad_key_en=vocab_en['<pad>']):
    counter_1 = 0
    counter_2 = batch_size_

    while True:
        max_length_en_batch = 0
        for seq in train_en_int_[counter_1:counter_2]:
            length_of_current_seq = len(seq)
            if length_of_current_seq >= max_length_en_batch:
                max_length_en_batch = length_of_current_seq

        max_length_de_batch = 0
        for seq in train_de_int_[counter_1:counter_2]:
            length_of_current_seq = len(seq)
            if length_of_current_seq >= max_length_de_batch:
                max_length_de_batch = length_of_current_seq
        
        # max_length=0
        # if max_length_en_batch >= max_length_de_batch:
        #     max_length = max_length_en_batch
        # else:
        #     max_length = max_length_de_batch


        batch_of_inputs  = np.full((batch_size_,max_length_de_batch), pad_key_de)
        batch_of_outputs = np.full((batch_size_,max_length_en_batch), pad_key_en)

        for i, seq in enumerate(train_de_int_[counter_1:counter_2]):
            batch_of_inputs[i,:len(seq)] = seq

        for i, seq in enumerate(train_en_int_[counter_1:counter_2]):
            batch_of_outputs[i,:len(seq)] = seq

        yield batch_of_inputs, batch_of_outputs

        counter_1 = counter_1 + batch_size_
        counter_2 = counter_2 + batch_size_

        if counter_2 >= len(train_en_int_):
            counter_1 = 0
            counter_2 = batch_size_
    

In [115]:
########################################

In [116]:
np.full((2,2), vocab_de['<pad>'])

array([[1, 1],
       [1, 1]])

In [117]:
########################################

In [118]:
batch_size = 64
gen = data_generator(train_en_int, train_de_int, vocab_en, vocab_de, batch_size, pad_key_de=vocab_de['<pad>'], pad_key_en=vocab_en['<pad>'])


In [119]:
a, b = next(gen)
a, b


(array([[  2,   4,   5, ...,   1,   1,   1],
        [  2,  16,   7, ...,   1,   1,   1],
        [  2,  20,  21, ...,   1,   1,   1],
        ...,
        [  2, 286, 287, ...,   1,   1,   1],
        [  2,  12,   0, ...,   1,   1,   1],
        [  2,  20,   5, ...,   1,   1,   1]]),
 array([[  2,   4,   5, ...,   1,   1,   1],
        [  2,  14,  15, ...,   1,   1,   1],
        [  2,  20,  24, ...,   1,   1,   1],
        ...,
        [  2, 299, 300, ...,   1,   1,   1],
        [  2,  40,   5, ...,   1,   1,   1],
        [  2,  20,   5, ...,   1,   1,   1]]))

In [120]:
a, b = next(gen)
a, b


(array([[  2,  46,  66, ..., 172,   3,   1],
        [  2,  20,  27, ...,   1,   1,   1],
        [  2,  20,  92, ...,   1,   1,   1],
        ...,
        [  2,   4, 101, ...,   1,   1,   1],
        [  2,  46, 123, ...,   1,   1,   1],
        [  2,  46,  66, ...,   1,   1,   1]]),
 array([[  2,  61,  16, ...,   1,   1,   1],
        [  2,  20,  30, ...,   1,   1,   1],
        [  2,  20,  89, ...,   1,   1,   1],
        ...,
        [  2,   4,  97, ...,   1,   1,   1],
        [  2,  20, 241, ...,   1,   1,   1],
        [  2,  20,  61, ...,   1,   1,   1]]))

In [121]:
print(train_de_int[0])
print(train_de_int[64])

[2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 3]
[2, 46, 66, 11, 33, 162, 299, 48, 64, 248, 20, 32, 28, 300, 0, 301, 32, 64, 0, 103, 172, 3]


In [122]:
a.shape


(64, 23)

In [64]:
b.shape


(64, 21)

In [127]:
a.T.shape

(23, 64)

# Model

In [132]:
# # encoder
# # WITHOUT DROPOUT
# # input_size: size of german vocabulary, cuz german vocab encoder ko pass krain gay
# class Encoder(nn.Module):
#     def __init__(self, input_size, embedding_size, hidden_size, num_layers):
#         super(Encoder, self).__init__()
# #        self.dropout = nn.Dropout(p)
#         self.hidden_size = hidden_size
#         self.num_layers = num_layers

#         self.embedding = nn.Embedding(input_size, embedding_size)
#         self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers)

#     def forward(self, x):
#         # x shape: (seq_length, N) where N is batch size

# #        embedding = self.dropout(self.embedding(x))
#         embedding = self.embedding(x)
#         # embedding shape: (seq_length, N, embedding_size)

#         outputs, (hidden, cell) = self.rnn(embedding)       # hidden_state and cell_
#         # outputs shape: (seq_length, N, hidden_size)

#         return hidden, cell


# with dropout
# input_size: size of german vocabulary, cuz german vocab encoder ko pass krain gay
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, p):
        super(Encoder, self).__init__()
        self.dropout = nn.Dropout(p)
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=p)

    def forward(self, x):
        # x shape: (seq_length, N) where N is batch size

        embedding = self.dropout(self.embedding(x))
        # embedding shape: (seq_length, N, embedding_size)

        outputs, (hidden, cell) = self.rnn(embedding)
        # outputs shape: (seq_length, N, hidden_size)

        return hidden, cell



In [133]:
# Decoder

# # # WITHOUT dropout
# # make sure hidden_size of encoder and decoder are the same
# # input_size: size of english vocabulary
# # also note, output_size will be same as input_size, we are gonna output, soem sort of vector which corresponds to values for each word which is in our vacbulary 
# # if we have 10,000 words in our eng voca, our output for each word will be 10,000 dimensional, with prob for each word from vocab
# class Decoder(nn.Module):
#     def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers):
#         super(Decoder, self).__init__()
#         self.hidden_size = hidden_size
#         self.num_layers  = num_layers

# #        self.dropout   = nn.Dropout(p)
#         self.embedding = nn.Embedding(input_size, embedding_size)
#         self.rnn       = nn.LSTM(embedding_size, hidden_size, num_layers)
#         self.fc        = nn.Linear(hidden_size, output_size) # output_size also equal to input_size of decoder which is equal to eng vocab size

#     def forward(self, x, hidden, cell):
#         # shape of x: (N) but we want (1,N), cuz decoder will predict one word at a time, so given the precious hidden and cell state and the previous predicted word from the deocder, i.e. previous output (I guess) or ouput converted into proper word form
#         # (1,N): 1 represents we have a singlw word, we have N batches of a single word. yaar daikho, aap khud hi jab model kaam kr raha ho, andaza lga sktay ho k kia ho raha hay, ballpoint say diagram sketch kr lena
#         # N batches of a single word at the same time. this is in contrast to the encoder, where we had a sequence of words at a time
#         # so to encoder we can send the entire sequence, but when we are doing prediction, we are doing one at a time...

#         x = x.unsqueeze(0)

#         embedding = self.embedding(x)
# #        embedding = self.dropout(self.embedding(x))
#         # embedding shape: (1, N, embedding_size)

#         outputs, (hidden, cell) = self.rnn(embedding, (hidden, cell))
#         # shape of outputs: (1, N, hiddeen_size)

#         predictions = self.fc(outputs)
#         # shape of predictions: (1, N, length_of_vocab)

#         predictions = predictions.squeeze(0)

#         return predictions, hidden, cell






# with dropout
# make sure hidden_size of encoder and decoder are the same
# input_size: size of english vocabulary
# also note, output_size will be same as input_size, we are gonna output, soem sort of vector which corresponds to values for each word which is in our vacbulary 
# if we have 10,000 words in our eng voca, our output for each word will be 10,000 dimensional, with prob for each word from vocab
class Decoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size, num_layers, p):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers  = num_layers

        self.dropout   = nn.Dropout(p)
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn       = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=p)
        self.fc        = nn.Linear(hidden_size, output_size) # output_size also equal to input_size of decoder which is equal to eng vocab size

    def forward(self, x, hidden, cell):
        # shape of x: (N) but we want (1,N), cuz decoder will predict one word at a time, so given the precious hidden and cell state and the previous predicted word from the deocder, i.e. previous output (I guess) or ouput converted into proper word form
        # (1,N): 1 represents we have a singlw word, we have N batches of a single word. yaar daikho, aap khud hi jab model kaam kr raha ho, andaza lga sktay ho k kia ho raha hay, ballpoint say diagram sketch kr lena
        # N batches of a single word at the same time. this is in contrast to the encoder, where we had a sequence of words at a time
        # so to encoder we can send the entire sequence, but when we are doing prediction, we are doing one at a time...

        x = x.unsqueeze(0)

        embedding = self.dropout(self.embedding(x))
        # embedding shape: (1, N, embedding_size)

        outputs, (hidden, cell) = self.rnn(embedding, (hidden, cell))
        # shape of outputs: (1, N, hiddeen_size)

        predictions = self.fc(outputs)
        # shape of predictions: (1, N, length_of_vocab)

        predictions = predictions.squeeze(0)

        return predictions, hidden, cell


In [134]:
# mine
# class Seq2Seq(nn.Module):
#     def __init__(self, encoder, decoder):
#         super(Seq2Seq, self).__init__
#         self.encoder = encoder
#         self.decoder = decoder

#     def forward(self, source, target, teacher_force_ratio=0.5):
#         # we won't use teacher_force_ratio=1, i.e. the correct target words all the time, cuz then at test time the words that it predicts might be completely different than what it sees at training time
#         batch_size = source.shape[1]
#         target_len = target.shape[0]
#         target_vocab_size = len(vocab_en)

#         outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(debice)

#         hidden, cell = self.encoder(source)

#         # Grab start token
#         x = target[0]

#         # loop k andar word dar word predictions lenay lgay hain decoder say
#         for t in  range(1, target_len):
#             output, hidden, cell = self.decoder(x, hidden, cell)
#             # output (N, eng_vocab_size): for each batch id 
#             outputs[t] = output

#             best_guess = ouput.argmax(1)

#             # yani exact-target hoga 50% of the times and and predicted-target hoga 50% of the times 
#             x = target[t] if random.random() < teacher_force_ratio else best_guess
#             # next ye 'x' use hoga, loss calculations k liye
#         return outputs


# his
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, source, target, teacher_force_ratio=0.5):
        batch_size = source.shape[1]
        target_len = target.shape[0]
        target_vocab_size = len(vocab_en)

        outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)

        hidden, cell = self.encoder(source)

        # Grab the first input to the Decoder which will be <SOS> token
        x = target[0]

        for t in range(1, target_len):
            # Use previous hidden, cell as context from encoder at start
            output, hidden, cell = self.decoder(x, hidden, cell)

            # Store next output prediction
            outputs[t] = output

            # Get the best word the Decoder predicted (index in the vocabulary)
            best_guess = output.argmax(1)

            # With probability of teacher_force_ratio we take the actual next word
            # otherwise we take the word that the Decoder predicted it to be.
            # Teacher Forcing is used so that the model gets used to seeing
            # similar inputs at training and testing time, if teacher forcing is 1
            # then inputs at test time might be completely different than what the
            # network is used to. This was a long comment.
            x = target[t] if random.random() < teacher_force_ratio else best_guess

        return outputs

# Training


In [135]:
# Training Hyperparameters
num_epochs = 10
learning_rate = 0.001
batch_size=128

# Model Hyperparameters
#load_model = False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_size_encoder = len(vocab_de)
input_size_decoder = len(vocab_en)
output_size        = len(vocab_en)

encoder_embedding_size = 256
decoder_embedding_size = 256
hidden_size            = 512

num_layers = 2
encoder_dropout = 0.5
decoder_dropout = 0.5

# # Tensorboard
# writer = SummaryWriter(f'runs/Loss_plot')
# # see video on Tensorboard of Aladdin Persson youtube channel for more functionalities of tensorboard
step = 0

train_iterator = data_generator(train_en_int, train_de_int, vocab_en, vocab_de, batch_size, pad_key_de=vocab_de['<pad>'], pad_key_en=vocab_en['<pad>'])

# encoder initialize kia
encoder_net= Encoder(input_size_encoder, encoder_embedding_size,
                    hidden_size, num_layers, encoder_dropout).to(device)
# encoder_net= Encoder(input_size_encoder, encoder_embedding_size,
#                     hidden_size, num_layers).to(device)



# decoder alehda initialize kia
decoder_net= Decoder(input_size_decoder, decoder_embedding_size,
                    hidden_size, output_size, num_layers, decoder_dropout).to(device)
# decoder_net= Decoder(input_size_decoder, decoder_embedding_size,
#                     hidden_size, output_size, num_layers).to(device)



# ab model initialize krnay lgay hain

# classes send nahi keen, bal k, initialized objects pass kiye hain
model = Seq2Seq(encoder_net, decoder_net).to(device)

pad_idx   = vocab_en['<pad>']
criterion  = nn.CrossEntropyLoss(ignore_index=pad_idx) #######################################################################
optimizer = optim.Adam(model.parameters(), lr=learning_rate)



# if load_model:
#     load_checkpoint(torch.load('my_checkpoint.pth.ptar'), model, optimizer.state_dict())







for epoch in range(num_epochs):
    print(f'Epoch [{epoch} / {num_epochs}]')

    # checkpoint = {'state_dict': model.state_dict(), 'optimizer': optimizer.state_dict()}
    # save_checkpoint(checkpoint)

    running_loss = 0
    
    for batch_idx, (batch_of_inputs, batch_of_outputs) in enumerate(train_iterator):
        inp_data = torch.from_numpy(batch_of_inputs.T).clone().long().to(device)
        target   = torch.from_numpy(batch_of_outputs.T).clone().long().to(device)

        output = model(inp_data, target)
        # output shape: (trg_len, batch_size, output_dim)

        output = output[1:].reshape(-1, output.shape[2])  # start token send nahi kar rahay [1:]
        target = target[1:].reshape(-1)

        optimizer.zero_grad()
        loss = criterion(output, target)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)  # exploding gradients ?

        optimizer.step()

        # writer.add_scalar('Training loss', loss, global_step=step)
        step+=1

        running_loss = running_loss+loss.item()
        avg_running_loss = running_loss/(batch_idx+1)
        print()
        print(f'{batch_idx+1}: ', avg_running_loss)
        print()

        if (batch_idx+1) >= (len(train_de_int)//batch_size):
            break


Epoch [0 / 10]

1:  8.716712951660156


2:  8.681222915649414


3:  8.599066416422525


4:  8.41355037689209


5:  8.141777420043946


6:  7.883374214172363


7:  7.659175940922329


8:  7.4384713768959045


9:  7.230960792965359


10:  7.059000825881958


11:  6.925720518285578


12:  6.805922269821167


13:  6.712966295389029


14:  6.625096934182303


15:  6.549421882629394


16:  6.4886274337768555


17:  6.425493184257956


18:  6.372638834847344


19:  6.327373805798982


20:  6.282622504234314


21:  6.243214016868954


22:  6.207907633347944


23:  6.1740614849588145


24:  6.1429585218429565


25:  6.115986385345459


26:  6.08501700254587


27:  6.060308032565647


28:  6.036690558706011


29:  6.010154181513293


30:  5.987142658233642


31:  5.968896927372102


32:  5.948457270860672


33:  5.9308279499863135


34:  5.91301630525028


35:  5.894272381918771


36:  5.87937859694163


37:  5.861010113277951


38:  5.845021172573692


39:  5.8279601855155745


40:  5.814984738

KeyboardInterrupt: 

# Making Translations


In [136]:
def translate_sentence(model_, sentence_,  vocab_en_, vocab_de_, index_to_word_en_, device_, max_length=20):
    tokenized_sentence     = ['<sos>'] + nltk.word_tokenize(sentence_.lower(), language='german') + ['<eos>']
    tokenized_sentence_int = word_seq_to_int_seq(tokenized_sentence, vocab_de_)
    sentence_tensor        = torch.LongTensor(tokenized_sentence_int).unsqueeze(1).to(device_)

    with torch.no_grad():
        hidden, cell = model.encoder(sentence_tensor)
    outputs = [vocab_en_['<sos>']]

    for _ in range(max_length):
        previous_word = torch.LongTensor([outputs[-1]]).to(device_)  # outputs ka aakhri word
    
        with torch.no_grad():
            output, hidden, cell = model.decoder(previous_word, hidden, cell)  # yani hidden and cell aik hi bar calculate ho k, encoder say aa gaye
            best_guess = output.argmax(1).item()                               # notice 'ARG'max
        outputs.append(best_guess)
    
         # Model predicts it's the end of the sentence
        if output.argmax(1).item() == vocab_en_["<eos>"]:                       # end of sequnce token predict hua, tu loop break ho jaye gi
            break
    
    translated_sentence = [index_to_word_en_[idx] for idx in outputs]
    return translated_sentence[1:]

    

In [137]:
max_length=20

sentence = "Ein Mann mit langem Bart"
tokenized_sentence = ['<sos>'] + nltk.word_tokenize(sentence.lower(), language='german') + ['<eos>']
tokenized_sentence_int = word_seq_to_int_seq(tokenized_sentence, vocab_de)

sentence_tensor = torch.LongTensor(tokenized_sentence_int).unsqueeze(1).to(device)
sentence_tensor

tensor([[   2],
        [  20],
        [  27],
        [  17],
        [1453],
        [ 339],
        [   3]], device='cuda:0')

In [138]:
with torch.no_grad():
    hidden, cell = model.encoder(sentence_tensor)

outputs = [vocab_en['<sos>']]
hidden, cell, outputs

(tensor([[[ 2.0603e-01, -5.0115e-01, -1.4011e-01,  ...,  5.6723e-01,
           -2.8295e-01,  2.1163e-02]],
 
         [[ 5.3847e-01,  2.8030e-01,  1.5707e-04,  ...,  5.3573e-01,
            7.1478e-04,  1.6804e-02]]], device='cuda:0'),
 tensor([[[ 0.2132, -0.5710, -0.2713,  ...,  0.6614, -1.1137,  2.4450]],
 
         [[ 0.6154,  1.7204,  0.0166,  ...,  4.6864,  0.0506,  0.5383]]],
        device='cuda:0'),
 [2])

In [139]:
for _ in range(max_length):
    previous_word = torch.LongTensor([outputs[-1]]).to(device)  # outputs ka aakhri word

    with torch.no_grad():
        output, hidden, cell = model.decoder(previous_word, hidden, cell)  # yani hidden and cell aik hi bar calculate ho k, encoder say aa gaye
        best_guess = output.argmax(1).item()                               # notice 'ARG'max
    outputs.append(best_guess)

     # Model predicts it's the end of the sentence
    if output.argmax(1).item() == vocab_en["<eos>"]:                       # end of sequnce token predict hua, tu loop break ho jaye gi
        break

translated_sentence = [index_to_word_en[idx] for idx in outputs]
print(translated_sentence)
len(translated_sentence)

    
        

['<sos>', 'a', 'man', 'in', 'a', 'blue', 'shirt', '<eos>']


8

In [140]:
sentence = "Ein Mann mit langem Bart"
translated_sentence = translate_sentence(model, sentence,  vocab_en, vocab_de, index_to_word_en, device, max_length=20)
print(translated_sentence), len(translated_sentence)


['a', 'man', 'in', 'a', 'hat', '<eos>']


(None, 6)

In [141]:
sentence = 'zwei'
translated_sentence = translate_sentence(model, sentence,  vocab_en, vocab_de, index_to_word_en, device, max_length=20)
print(translated_sentence), len(translated_sentence)


['bodies', 'russian', 'are', 'a', 'a', '<eos>']


(None, 6)

In [142]:
sentence = 'Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche'
translated_sentence = translate_sentence(model, sentence,  vocab_en, vocab_de, index_to_word_en, device, max_length=20)
print(translated_sentence), len(translated_sentence)


['two', 'young', 'men', 'are', 'in', 'the', 'street', 'in', 'a', '<eos>']


(None, 10)